In [1]:
# 这个notebook 介绍了 如何对split learning 发起 inverse-model attack攻击

In [3]:
# 导包
import sys
sys.path.append('/home/yangchenxi/data/PP-Split')
from ppsplit.attacks.model_inversion.inverse_model import InverseModelAttack
from ppsplit.utils.utils import create_dir
import torch
import os


# cifar10 （图像多分类）

In [4]:
# 导包和超参数设置
from target_model.data_preprocessing.preprocess_cifar10 import get_cifar10_normalize,get_one_data,deprocess

from target_model.models.VGG import VGG,VGG5Decoder,model_cfg
from target_model.models.splitnn_utils import split_weights_client

test_num = 2 # 测试编号（对应结果文件夹名称）
split_layer = 2 # 模型切割点 （split point）在该层之前的层（含），作为client的模型，之后的层作为server的模型

# 重要路径设置
unit_net_route = '/data/dengruijun/project/AISecurity/Inverse_efficacy/results/VGG5/BN+Tanh/2-20240101/VGG5-params-19ep.pth'
results_dir = f'../results/VGG5/{test_num}/'
inverse_dir = results_dir + 'layer'+str(split_layer)+'/'
decoder_net_route = results_dir + f'Decoder-layer{split_layer}.pth' # 攻击的decoder net存储位置


In [5]:
# 准备基本模型client net
# split_layer_list = list(range(len(model_cfg['VGG5']))) # 可能的切割点

# 创建对应文件夹
create_dir(results_dir)
create_dir(inverse_dir)

# 把unit模型切割成client-server 的模型pair
client_net = VGG('Client','VGG5',split_layer,model_cfg)
pweights = torch.load(unit_net_route)
if split_layer < len(model_cfg['VGG5']):
    pweights = split_weights_client(pweights,client_net.state_dict())
client_net.load_state_dict(pweights)


features.0.weight
features.0.bias
features.1.weight
features.1.bias
features.1.running_mean
features.1.running_var
features.1.num_batches_tracked
features.4.weight
features.4.bias
features.5.weight
features.5.bias
features.5.running_mean
features.5.running_var
features.5.num_batches_tracked


<All keys matched successfully>

In [6]:
# 准备inverse_model attack使用到的东西
# 创建Inverse Model Attack对象
im_attack = InverseModelAttack(decoder_route=decoder_net_route,data_type=1,inverse_dir=inverse_dir)

# 加载decoder模型
if os.path.isfile(decoder_net_route): # 如果已经训练好了
    print("=> loading decoder model '{}'".format(decoder_net_route))
    decoder_net = torch.load(decoder_net_route)
else: # 如果没有
    print("train decoder model...")
    decoder_net = VGG5Decoder(split_layer=split_layer)
    # 训练decoder
    trainloader,testloader = get_cifar10_normalize(batch_size=32)

    decoder_net= im_attack.train_decoder(client_net=client_net,decoder_net=decoder_net,
                            train_loader=trainloader,test_loader=testloader,
                            epochs=20)


train decoder model...
----train decoder----
client_net: 
VGG(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Tanh()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Tanh()
  )
  (denses): Sequential()
)
decoder_net: 
VGG5Decoder(
  (features): Sequential(
    (0): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Tanh()
    (3): ConvTranspose2d(32, 3, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
    (4): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): T

100%|██████████| 1563/1563 [00:20<00:00, 76.34it/s]


--- epoch: 0, train_loss: [0.029827436432242393]
Epoch 1


100%|██████████| 1563/1563 [00:11<00:00, 130.74it/s]


--- epoch: 1, train_loss: [0.026516055688261986]
Epoch 2


...

--- epoch: 17, train_loss: [0.02248193882405758]
Epoch 18


100%|██████████| 1563/1563 [00:14<00:00, 104.50it/s]


--- epoch: 18, train_loss: [0.0224134624004364]
Epoch 19


100%|██████████| 1563/1563 [00:16<00:00, 92.41it/s] 


--- epoch: 19, train_loss: [0.02234969101846218]
model saved


In [7]:
# 实现攻击,恢复testloader中所有图片
trainloader,testloader = get_cifar10_normalize(batch_size=1)

im_attack.inverse(client_net=client_net,decoder_net=decoder_net,
                  train_loader=trainloader,test_loader=testloader,
                  deprocess=deprocess,
                  save_fake=True)

----train decoder----
client_net: 
VGG(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Tanh()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Tanh()
  )
  (denses): Sequential()
)
decoder_net: 
VGG5Decoder(
  (features): Sequential(
    (0): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Tanh()
    (3): ConvTranspose2d(32, 3, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
    (4): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): Tanh()
  )
  (denses): S

100%|██████████| 10000/10000 [01:06<00:00, 150.70it/s]


SSIM: 0.8238943966925144,              MSE:0.0906554913601838
average time: 0.003288646101951599 avg infer time:0.0006499203443527221


# Bank数据集 （表格数据二分类）

In [12]:
# 导包和超参数设置
from target_model.data_preprocessing.preprocess_bank import preprocess_bank

from target_model.models.BankNet import BankNet1,BankNetDecoder1,bank_cfg
from target_model.models.splitnn_utils import split_weights_client

test_num = 6 # 测试编号（对应结果文件夹名称）
split_layer = 2 # 模型切割点 （split point）在该层之前的层（含），作为client的模型，之后的层作为server的模型

# 重要路径设置
unit_net_route = '/data/dengruijun/FinTech/PP-Split/results/trained_models/Bank/bank-20ep_params.pth'
results_dir = f'../results/Bank/{test_num}/'
inverse_dir = results_dir + 'layer'+str(split_layer)+'/'
decoder_net_route = results_dir + f'Decoder-layer{split_layer}.pth' # 攻击的decoder net存储位置


In [13]:
# 准备target model的 client net（对模型进行切割）
create_dir(results_dir)
create_dir(inverse_dir)

client_net = BankNet1(layer=split_layer)
pweights = torch.load(unit_net_route)
if split_layer < len(bank_cfg):
    pweights = split_weights_client(pweights,client_net.state_dict())
client_net.load_state_dict(pweights)



linear1.weight
linear1.bias
linear2.weight
linear2.bias


<All keys matched successfully>

In [8]:
# 准备inverse_model attack使用到的东西
# 创建Inverse Model Attack对象
im_attack = InverseModelAttack(decoder_route=decoder_net_route,data_type=0,inverse_dir=inverse_dir)

# 加载decoder模型
if os.path.isfile(decoder_net_route): # 如果已经训练好了
    print("=> loading decoder model '{}'".format(decoder_net_route))
    decoder_net = torch.load(decoder_net_route)
else: # 如果没有
    print("train decoder model...")
    decoder_net = BankNetDecoder1(layer=split_layer)
    # 训练decoder
    trainloader,testloader = preprocess_bank(batch_size=32)

    decoder_net= im_attack.train_decoder(client_net=client_net,decoder_net=decoder_net,
                            train_loader=trainloader,test_loader=testloader,
                            epochs=20)

=> loading decoder model '../results/Bank/6/Decoder-layer2.pth'


In [9]:
# 实现攻击,恢复testloader中所有表格数据行
trainloader,testloader = preprocess_bank(batch_size=1)

im_attack.inverse(client_net=client_net,decoder_net=decoder_net,
                  train_loader=trainloader,test_loader=testloader,
                  save_fake=True)

===============processing data===============


  0%|          | 0/8238 [00:00<?, ?it/s]

X_train.shape: (32950, 63)
X_test.shape: (8238, 63)
y_train.shape: (32950, 1)
y_test.shape: (8238, 1) <class 'numpy.ndarray'>
===============processing data end===============
----train decoder----
client_net: 
BankNet1(
  (linear1): Linear(in_features=63, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=64, bias=True)
)
decoder_net: 
BankNetDecoder1(
  (delinear1): Linear(in_features=64, out_features=128, bias=True)
  (ReLU1): ReLU()
  (delinear2): Linear(in_features=128, out_features=63, bias=True)
)


100%|██████████| 8238/8238 [00:20<00:00, 397.55it/s]


cosine: 0.9167903578593969,               Euclidean: 1.294159001821137,              MSE:0.042359481014795926
average time: 0.0006233145525571819 avg infer time:0.0003061840388924788


## VFL

### data+model

In [19]:
# 导包
import sys
sys.path.append('/data/yangjirui/drj/PP-Split')
sys.path.append('/data/yangjirui/drj/PP-Split/ppsplit/')
from ppsplit.attacks.model_inversion.UIFV.UIFV_model_data import *
# from ppsplit.utils.utils import create_dir
# import torch
# import os

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

tab = {
    'boolList': [i for i in range(0, 22)],
    'onehot': {
        'marital': [0, 1, 2, 3], 'default': [4, 5, 6],
        'loan': [7, 8, 9], 'month': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
        'poutcome': [20, 21, 22]
    },
    'numList': [i for i in range(23, 28)]
}

ModuleNotFoundError: No module named 'ruamel'

In [6]:
# 设置超参数

parser = argparse.ArgumentParser("TabRebuild")
parser.add_argument('--multiple', action='store_true', help='Whether to conduct multiple experiments')
parser.add_argument('--name', type=str, default='decoder-rebuild', help='experiment name')
parser.add_argument('--data_dir', default='/home/yangjirui/data/vfl-tab-reconstruction/dataset/bank/bank-additional/bank-additional-full.csv',
                    help='location of the data corpus')
parser.add_argument('--batch_size', type=int, default=64, help='batch size')
parser.add_argument('--lr', type=float, default=0.001, help='init learning rate')
parser.add_argument('--eps', type=float, default=1e-3)
parser.add_argument('--workers', type=int, default=6, help='num of workers')
parser.add_argument('--k', type=int, default=2, help='num of client')
parser.add_argument('--seed', type=int, default=1, help='random seed')
parser.add_argument('--save',  default='/data/yangjirui/vfl-tab-reconstruction/rebuild-data/bank/v5/',
                    help='location of the data corpus')
parser.add_argument('--decoder_mode',
                    default="/data/yangjirui/vfl-tab-reconstruction/model/bank/v5/attack-model/model+data+.pth.tar",
                    help='location of the decoder mode')
parser.add_argument('--momentum', type=float, default=0.9, help='momentum')
# parser.add_argument('--inverse_data_output', default="/home/yangjirui/paper-code/data/adult/2layer-noloss/base-random/inverse_data.csv",
#                     help='location of the data corpus')
parser.add_argument('--weight_decay', type=float, default=3e-5, help='weight decay')
parser.add_argument('--gamma', type=float, default=0.97, help='weight decay')
parser.add_argument('--epochs', type=int, default=120, help='num of epochs')
parser.add_argument('--base_mode', default='/data/yangjirui/vfl-tab-reconstruction/model/bank/v5/best.pth.tar', type=str, metavar='PATH',
                    help='path to latest base mode (default: none)')
parser.add_argument('--grad_clip', type=float, default=5., help='gradient clipping')

args, unknown = parser.parse_known_args()

In [7]:
# 开始攻击
train, test = preprocess(args.data_dir)

acc, onehot_acc, num_acc, similarity, euclidean_dist = rebuild(train_data=train, test_data=test, tab=tab,
                                                               device=device, args=args)

print(f"acc:{acc}, onehot_acc:{onehot_acc}, num_acc:{num_acc}, similarity:{similarity}, euclidean_dist:{euclidean_dist}")

===============processing data===============
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null

100%|██████████| 515/515 [00:08<00:00, 60.34it/s]


acc: 0.9878375584322187
onehot_acc: 0.9970449029126213
num_acc: 0.9786302139518158
similarity 0.8835727652144364
euclidean_dist 1.3748186748004654
################################ save data ############################
[[0.9878375584322187 0.9970449029126213 0.9786302139518158
  0.8835727652144364 1.3748186748004654 False 'decoder-rebuild'
  '/home/yangjirui/data/vfl-tab-reconstruction/dataset/bank/bank-additional/bank-additional-full.csv'
  64 0.001 0.001 6 2 1
  '/data/yangjirui/vfl-tab-reconstruction/rebuild-data/bank/v5/'
  '/data/yangjirui/vfl-tab-reconstruction/model/bank/v5/attack-model/model+data+.pth.tar'
  0.9 3e-05 0.97 120
  '/data/yangjirui/vfl-tab-reconstruction/model/bank/v5/best.pth.tar' 5.0
  12]]
acc:0.9878375584322187, onehot_acc:0.9970449029126213, num_acc:0.9786302139518158, similarity:0.8835727652144364, euclidean_dist:1.3748186748004654


# Credit 数据集 （表格数据二分类）

In [3]:
# 导包和超参数设置
from target_model.data_preprocessing.preprocess_credit import preprocess_credit

from target_model.models.CreditNet import CreditNet1,CreditNetDecoder1,credit_cfg
from target_model.models.splitnn_utils import split_weights_client

test_num = 10 # 测试编号（对应结果文件夹名称）
split_layer = 3 # 模型切割点 （split point）在该层之前的层（含），作为client的模型，之后的层作为server的模型

# 重要路径设置
unit_net_route = '/home/yangchenxi/data/PP-Split/results/trained_models/credit/credit-20ep_params.pth'
results_dir = f'../results/Credit/{test_num}/'
inverse_dir = results_dir + 'layer'+str(split_layer)+'/' # 储存
decoder_net_route = results_dir + f'Decoder-layer{split_layer}.pth' # 攻击的decoder net存储位置

In [4]:
# 准备target model的 client net（对模型进行切割）
create_dir(results_dir)
create_dir(inverse_dir)

client_net = CreditNet1(layer=split_layer)
pweights = torch.load(unit_net_route)
if split_layer < len(credit_cfg):
    pweights = split_weights_client(pweights,client_net.state_dict())
client_net.load_state_dict(pweights)


linear1.weight
linear1.bias
batch_norm1.weight
batch_norm1.bias
batch_norm1.running_mean
batch_norm1.running_var
batch_norm1.num_batches_tracked
linear2.weight
linear2.bias


<All keys matched successfully>

In [5]:
# 准备inverse_model attack使用到的东西
# 创建Inverse Model Attack对象
im_attack = InverseModelAttack(decoder_route=decoder_net_route,data_type=0,inverse_dir=inverse_dir)

# 加载decoder模型
if os.path.isfile(decoder_net_route): # 如果已经训练好了
    print("=> loading decoder model '{}'".format(decoder_net_route))
    decoder_net = torch.load(decoder_net_route)
else: # 如果没有
    print("train decoder model...")
    decoder_net = CreditNetDecoder1(layer=split_layer)
    # optimizer = torch.optim.SGD(decoder_net.parameters(), 1e-3)

    # 训练decoder
    trainloader,testloader = preprocess_credit(batch_size=32)

    decoder_net= im_attack.train_decoder(client_net=client_net,decoder_net=decoder_net,
                            train_loader=trainloader,test_loader=testloader,
                            epochs=20,
                            # optimizer=optimizer
                            )

=> loading decoder model '../results/Credit/10/Decoder-layer3.pth'


In [6]:
# 实现攻击,恢复testloader中所有表格数据行
trainloader,testloader = preprocess_credit(batch_size=1)

im_attack.inverse(client_net=client_net,decoder_net=decoder_net,
                  train_loader=trainloader,test_loader=testloader,
                  save_fake=True)

===============processing data===============


  0%|          | 0/61503 [00:00<?, ?it/s]

X_train.shape: (246008, 250)
X_test.shape: (61503, 250)
y_train.shape: (246008, 1)
y_test.shape: (61503, 1) <class 'numpy.ndarray'>
===============processing data end===============
----train decoder----
client_net: 
CreditNet1(
  (linear1): Linear(in_features=250, out_features=512, bias=True)
  (batch_norm1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (ReLU1): LeakyReLU(negative_slope=0.01)
  (linear2): Linear(in_features=512, out_features=128, bias=True)
)
decoder_net: 
CreditNetDecoder1(
  (delinear1): Linear(in_features=128, out_features=512, bias=True)
  (ReLU1): ReLU()
  (delinear2): Linear(in_features=512, out_features=250, bias=True)
)


100%|██████████| 61503/61503 [01:49<00:00, 562.62it/s] 


cosine: 0.9869267164826434,               Euclidean: 1.4236435810680315,              MSE:0.009183392138517262
average time: 0.00035009122411578644 avg infer time:0.00031623337855465504


# Purchase100 数据集 （表格数据多分类）

In [2]:
# 导包和超参数设置
from target_model.data_preprocessing.preprocess_purchase import preprocess_purchase

from target_model.models.PurchaseNet import PurchaseClassifier1,PurchaseDecoder1,purchase_cfg
from target_model.models.splitnn_utils import split_weights_client

test_num = 1.1 # 测试编号（对应结果文件夹名称）
split_layer = 3 # 模型切割点 （split point）在该层之前的层（含），作为client的模型，之后的层作为server的模型

# 重要路径设置
unit_net_route = '/home/yangchenxi/data/PP-Split/results/trained_models/Purchase100/Purchase_bestmodel_param.pth'
results_dir = f'../results/Purchase/{test_num}/'
inverse_dir = results_dir + 'layer'+str(split_layer)+'/'
decoder_net_route = results_dir + f'Decoder-layer{split_layer}.pth' # 攻击的decoder net存储位置


In [3]:
# 准备target model的 client net（对模型进行切割）
create_dir(results_dir)
create_dir(inverse_dir)

client_net = PurchaseClassifier1(layer=split_layer)
pweights = torch.load(unit_net_route)
if split_layer < len(purchase_cfg):
    pweights = split_weights_client(pweights,client_net.state_dict())
client_net.load_state_dict(pweights)



linear1.weight
linear1.bias
linear2.weight
linear2.bias


<All keys matched successfully>

In [4]:
# 准备inverse_model attack使用到的东西
# 创建Inverse Model Attack对象
im_attack = InverseModelAttack(decoder_route=decoder_net_route,data_type=0,inverse_dir=inverse_dir)

# 加载decoder模型
if os.path.isfile(decoder_net_route): # 如果已经训练好了
    print("=> loading decoder model '{}'".format(decoder_net_route))
    decoder_net = torch.load(decoder_net_route)
else: # 如果没有
    print("train decoder model...")
    decoder_net = PurchaseDecoder1(layer=split_layer)
    # 训练decoder
    trainloader,testloader = preprocess_purchase(batch_size=32)

    decoder_net= im_attack.train_decoder(client_net=client_net,decoder_net=decoder_net,
                            train_loader=trainloader,test_loader=testloader,
                            epochs=20)

train decoder model...
[('Tanh',), ('D', 1024, 512), ('Tanh',), ('D', 600, 1024)]
purchase100 dataset processing...
datset route: /home/dengruijun/data/FinTech/DATASET/kaggle-dataset/Purchase100//data.npz
original dataset shape:  (197324, 600)
After random selection, dataset shape:  (197324, 600)
After split between classifier and attack: 
training dataset shape:  (157859, 600)
testing dataset shape:  (39465, 600)


  0%|          | 0/4933 [00:00<?, ?it/s]

Data loading finished
----train decoder----
client_net: 
PurchaseClassifier1(
  (linear1): Linear(in_features=600, out_features=1024, bias=True)
  (Tanh1): Tanh()
  (linear2): Linear(in_features=1024, out_features=512, bias=True)
  (Tanh2): Tanh()
)
decoder_net: 
PurchaseDecoder1(
  (delinear1): Linear(in_features=512, out_features=1024, bias=True)
  (Tanh1): Tanh()
  (delinear2): Linear(in_features=1024, out_features=600, bias=True)
  (Tanh2): Tanh()
)
Epoch 0


  0%|          | 0/4933 [00:00<?, ?it/s]

--- epoch: 0, train_loss: [0.036627549678087234]
Epoch 1


...

--- epoch: 17, train_loss: [0.032218173146247864]
Epoch 18


  0%|          | 0/4933 [00:00<?, ?it/s]

--- epoch: 18, train_loss: [0.03365800902247429]
Epoch 19


100%|██████████| 4933/4933 [00:24<00:00, 203.06it/s]

--- epoch: 19, train_loss: [0.033522527664899826]
model saved


In [5]:
# 实现攻击,恢复testloader中所有表格数据行
trainloader,testloader = preprocess_purchase(batch_size=1)

im_attack.inverse(client_net=client_net,decoder_net=decoder_net,
                  train_loader=trainloader,test_loader=testloader,
                  save_fake=True)

purchase100 dataset processing...
datset route: /home/dengruijun/data/FinTech/DATASET/kaggle-dataset/Purchase100//data.npz
original dataset shape:  (197324, 600)
After random selection, dataset shape:  (197324, 600)
After split between classifier and attack: 
training dataset shape:  (157859, 600)
testing dataset shape:  (39465, 600)


  0%|          | 0/39465 [00:00<?, ?it/s]

Data loading finished
----train decoder----
client_net: 
PurchaseClassifier1(
  (linear1): Linear(in_features=600, out_features=1024, bias=True)
  (Tanh1): Tanh()
  (linear2): Linear(in_features=1024, out_features=512, bias=True)
  (Tanh2): Tanh()
)
decoder_net: 
PurchaseDecoder1(
  (delinear1): Linear(in_features=512, out_features=1024, bias=True)
  (Tanh1): Tanh()
  (delinear2): Linear(in_features=1024, out_features=600, bias=True)
  (Tanh2): Tanh()
)


100%|██████████| 39465/39465 [01:33<00:00, 420.57it/s]


cosine: 0.9504989085429434,               Euclidean: 4.34348811342676,              MSE:0.03172383672123201
average time: 0.000556240434597068 avg infer time:0.0003523296538405767


: 